<a href="https://colab.research.google.com/github/ugurcun/Elma_Agaclarindaki_Yaprak-Hastaliklarinin_Belirlenmesi/blob/main/Elma_A%C4%9Fa%C3%A7lar%C4%B1ndaki_Yaprak_Hastal%C4%B1klar%C4%B1n%C4%B1n_Belirlenmesi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

# Verileri okuma
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
sample_submission = pd.read_csv('sample_submission.csv')

In [23]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

# Verileri okuma
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
sample_submission = pd.read_csv('sample_submission.csv')


# Veri Ön İşleme

# 1. Eksik Değerleri Ele Alın
# Eksik değerleri sütun ortalaması veya medyanı ile doldur
for column in train_data.columns:
    if train_data[column].isnull().any():
        if pd.api.types.is_numeric_dtype(train_data[column]):
            train_data[column].fillna(train_data[column].mean(), inplace=True)
        else:
            train_data[column].fillna(train_data[column].mode()[0], inplace=True)

for column in test_data.columns:
    if test_data[column].isnull().any():
        if pd.api.types.is_numeric_dtype(test_data[column]):
            test_data[column].fillna(test_data[column].mean(), inplace=True)
        else:
            test_data[column].fillna(test_data[column].mode()[0], inplace=True)


# 2. Kategorik Değişkenleri Kodlama
# One-Hot Encoding veya Label Encoding kullanın
train_data = pd.get_dummies(train_data, drop_first=True)
test_data = pd.get_dummies(test_data, drop_first=True)


# 3. Outlier'ları Ele Alın (Opsiyonel)
# Outlier'ları tespit edip, verileri düzenleyin
# Örneğin, IQR (Interquartile Range) kullanarak outlier'ları tespit edin ve bu outlier'ları verinin ortanca değeri ile değiştirin
for column in train_data.select_dtypes(include=np.number).columns:
    Q1 = train_data[column].quantile(0.25)
    Q3 = train_data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    train_data[column] = np.where((train_data[column] < lower_bound) | (train_data[column] > upper_bound),
                                train_data[column].median(), train_data[column])


# Veri Bölme
from sklearn.model_selection import train_test_split

X = train_data.drop('healthy', axis=1)
y = train_data['healthy']
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)


# Model Seçimi ve Eğitimi
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error


# Model 1: Linear Regression
model_lr = LinearRegression()
model_lr.fit(X_train, y_train)
y_pred_lr = model_lr.predict(X_val)
rmse_lr = np.sqrt(mean_squared_error(y_val, y_pred_lr))
print("Linear Regression RMSE:", rmse_lr)


# Model 2: Random Forest Regression
model_rf = RandomForestRegressor()
model_rf.fit(X_train, y_train)
y_pred_rf = model_rf.predict(X_val)
rmse_rf = np.sqrt(mean_squared_error(y_val, y_pred_rf))
print("Random Forest Regression RMSE:", rmse_rf)



# En iyi modeli seçin ve test verileri üzerinde değerlendirin
if rmse_lr < rmse_rf:
    best_model = model_lr
    print("En iyi model: Linear Regression")
else:
    best_model = model_rf
    print("En iyi model: Random Forest Regression")

y_pred_test = best_model.predict(X_test)
rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test))
print("Test RMSE:", rmse_test)

# Tahminleri dosyaya yazın
# Align the columns of test_data with X_train
test_data = test_data.reindex(columns=X_train.columns, fill_value=0)

predictions = best_model.predict(test_data)
sample_submission['healthy'] = predictions
sample_submission.to_csv('submission.csv', index=False)



Linear Regression RMSE: 0.21443482385731552
Random Forest Regression RMSE: 0.2344036154692477
En iyi model: Linear Regression
Test RMSE: 0.20699218646819806
